In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
btc=pd.read_csv('/kaggle/input/btc-data/btcusdt_1h_zelta.csv')
btc

,datetime,open,high,low,close,volume
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199
1,2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006
2,2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572
3,2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030
4,2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329
...,...,...,...,...,...,...
35203,2022-01-12 01:30:00,42972.04,43095.26,42692.19,42800.38,1219.601780
35204,2022-01-12 02:30:00,42797.62,42823.69,42643.74,42659.20,702.103800
35205,2022-01-12 03:30:00,42664.71,42776.14,42597.41,42713.13,561.859930
35206,2022-01-12 04:30:00,42713.12,42886.28,42633.97,42729.29,681.142010


The provided code defines a class called NBeatsBlock in TensorFlow/Keras. This class represents a block of the N-BEATS forecasting model, which is a neural network architecture designed for time series forecasting. The NBeatsBlock class defines a building block for the N-BEATS forecasting model, with the ability to take an input time series, apply a series of hidden layers, and output both backcast and forecast components. This block can be used as a part of a larger N-BEATS architecture for time series prediction tasks.

In [ ]:
WINDOW_SIZE=6
HORIZON=1

class NBeatsBlock(tf.keras.layers.Layer):
  def __init__(self,
               input_size: int,
               theta_size: int,
               horizon: int,
               n_neurons: int,
               n_layers: int,
               **kwargs):
    super().__init__(**kwargs)
    self.input_size = input_size
    self.theta_size = theta_size
    self.horizon = horizon
    self.n_neurons = n_neurons
    self.n_layers = n_layers


    self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]

    self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

  def call(self, inputs):
    x = inputs
    for layer in self.hidden:
      x = layer(x)
    theta = self.theta_layer(x)

    backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]
    return backcast, forecast

# DataFrame Preparation:

**bitcoin_prices**: Creates a new DataFrame named bitcoin_prices using the 'close' column from a DataFrame btc.
Renames the 'close' column to 'Price' in the bitcoin_prices DataFrame.
# Creating Lag Features:

**bitcoin_prices_nbeats:** Creates a copy of the bitcoin_prices DataFrame called bitcoin_prices_nbeats.
Utilizes a loop to create lag features (time-shifted versions of the 'Price' column) for the specified WINDOW_SIZE.
The loop iterates from 1 to WINDOW_SIZE, creating new columns named "Price+1", "Price+2", ..., "Price+WINDOW_SIZE" in bitcoin_prices_nbeats. Each new column contains the 'Price' values shifted by the corresponding number of periods.
# Data Cleaning:

**bitcoin_prices_nbeats.dropna():** Removes rows with missing values (NaN) resulting from the lagged features.
.head(): Displays the first few rows of the cleaned DataFrame.


In [ ]:
bitcoin_prices=pd.DataFrame(btc['close'])
bitcoin_prices.rename(columns={'close':'Price'},inplace=True)
bitcoin_prices_nbeats = bitcoin_prices.copy()
for i in range(WINDOW_SIZE):
  bitcoin_prices_nbeats[f"Price+{i+1}"] = bitcoin_prices_nbeats["Price"].shift(periods=i+1)
bitcoin_prices_nbeats.dropna().head()

,Price,Price+1,Price+2,Price+3,Price+4,Price+5,Price+6
6,13780.41,13558.99,13601.01,13410.03,13330.18,13203.06,13529.01
7,13570.35,13780.41,13558.99,13601.01,13410.03,13330.18,13203.06
8,13499.99,13570.35,13780.41,13558.99,13601.01,13410.03,13330.18
9,13616.99,13499.99,13570.35,13780.41,13558.99,13601.01,13410.03
10,13570.01,13616.99,13499.99,13570.35,13780.41,13558.99,13601.01


In [ ]:
X = bitcoin_prices_nbeats.dropna().drop("Price", axis=1)
y = bitcoin_prices_nbeats.dropna()["Price"]


split_size = int(len(X) * 0.8)
X_train, y_train = X[:split_size], y[:split_size]
X_test, y_test = X[split_size:], y[split_size:]
len(X_train), len(y_train), len(X_test), len(y_test)

(28161, 28161, 7041, 7041)

In [ ]:
train_features_dataset = tf.data.Dataset.from_tensor_slices(X_train)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(y_train)

features_dataset = tf.data.Dataset.from_tensor_slices(X)
labels_dataset = tf.data.Dataset.from_tensor_slices(y)

test_features_dataset = tf.data.Dataset.from_tensor_slices(X_test)
test_labels_dataset = tf.data.Dataset.from_tensor_slices(y_test)


train_dataset = tf.data.Dataset.zip((train_features_dataset, train_labels_dataset))
test_dataset = tf.data.Dataset.zip((test_features_dataset, test_labels_dataset))


BATCH_SIZE = 1024
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
features_dataset = features_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
labels_dataset = labels_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


train_dataset, test_dataset

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 6), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 6), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>)

In [ ]:


N_EPOCHS = 5000
N_NEURONS = 512
N_LAYERS = 4
N_STACKS = 30

INPUT_SIZE = WINDOW_SIZE * HORIZON
THETA_SIZE = INPUT_SIZE + HORIZON

INPUT_SIZE, THETA_SIZE

(6, 7)

# Initialization of N-BEATS Block Layer:

**nbeats_block_layer:** Instantiates an NBeatsBlock layer with specified parameters (e.g., INPUT_SIZE, THETA_SIZE, HORIZON, N_NEURONS, N_LAYERS) and assigns the name "InitialBlock."
# Model Architecture:

**stack_input:** Creates an input layer with the specified shape (INPUT_SIZE) named "stack_input."
Applies the nbeats_block_layer to the stack_input to obtain backcast and forecast.

**residuals:** Computes the residuals by subtracting backcast from stack_input.
Stacking Blocks:

Uses a loop to create additional N-BEATS blocks (NBeatsBlock) and stack them.
For each block, computes the block's backcast and block_forecast using the same parameters as the initial block.
Updates the residuals by subtracting the block's backcast.
Updates the overall forecast by adding the block's block_forecast.
Model Creation:

**model_7:** Constructs a Keras model with the defined architecture, taking stack_input as input and outputting the final forecast.
Compiles the model using the mean absolute error (mae) as the loss function, the Adam optimizer, and additional metrics such as mean squared error (mse).
# Model Training:

**model_7.fit:** Trains the model on the specified training dataset (train_dataset) for a given number of epochs (N_EPOCHS).
Includes validation data (test_dataset) for monitoring performance during training.
Utilizes early stopping and learning rate reduction callbacks to improve training efficiency.

In [ ]:
nbeats_block_layer = NBeatsBlock(input_size=INPUT_SIZE,
                                 theta_size=THETA_SIZE,
                                 horizon=HORIZON,
                                 n_neurons=N_NEURONS,
                                 n_layers=N_LAYERS,
                                 name="InitialBlock")


stack_input = layers.Input(shape=(INPUT_SIZE), name="stack_input")


backcast, forecast = nbeats_block_layer(stack_input)

residuals = layers.subtract([stack_input, backcast], name=f"subtract_00")


for i, _ in enumerate(range(N_STACKS-1)):
  backcast, block_forecast = NBeatsBlock(
      input_size=INPUT_SIZE,
      theta_size=THETA_SIZE,
      horizon=HORIZON,
      n_neurons=N_NEURONS,
      n_layers=N_LAYERS,
      name=f"NBeatsBlock_{i}"
  )(residuals)

  residuals = layers.subtract([residuals, backcast], name=f"subtract_{i}")
  forecast = layers.add([forecast, block_forecast], name=f"add_{i}")


model_7 = tf.keras.Model(inputs=stack_input,
                         outputs=forecast,
                         name="model_7_N-BEATS")


model_7.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=["mae", "mse"])


model_7.fit(train_dataset,
            epochs=N_EPOCHS,
            validation_data=test_dataset,
            verbose=0,
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=200, restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=100, verbose=1)])


Epoch 300: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 509: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 609: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.


In [ ]:
def make_pred(model,input_data):
    pseudo_forecast=model.predict(input_data)
    return tf.squeeze(pseudo_forecast)


In [ ]:
model_7_preds=make_pred(model_7,features_dataset)

35/35 [==============================] - 2s 16ms/step


# Model Evaluation

In [ ]:
def evaluate_model(y_true,y_pred):
    y_true=np.array(y_true)
    y_pred=np.array(y_pred)
    mae=tf.keras.metrics.mean_absolute_error(y_true,y_pred)
    mse=tf.keras.metrics.mean_squared_error(y_true,y_pred)
    rmse=np.sqrt(mse.numpy())
    mape=tf.keras.metrics.mean_absolute_percentage_error(y_true,y_pred)
    mase=mean_absolute_scaled_error(y_true,y_pred)
    return{'mae':mae.numpy(),
           'mse':mse.numpy(),
           'rmse':rmse,
           'mape':mape.numpy(),
           'mase':mase.numpy()}



In [ ]:
import tensorflow as tf
def mean_absolute_scaled_error(y_true,y_pred):
    mae=tf.reduce_mean(tf.abs(np.array(y_true)-np.array(y_pred)))
    mae_naive_noseason=tf.reduce_mean(tf.abs(np.array(y_true[1:])-np.array(y_true[:-1])))
    return mae/mae_naive_noseason

In [ ]:
import numpy as np
evaluate_model(y_true=y,y_pred=model_7_preds)

{'mae': 101.87207,
 'mse': 46614.246,
 'rmse': 215.90332,
 'mape': 0.50376546,
 'mase': 0.9872912091728135}

In [ ]:
pred=pd.Series(model_7_preds,index=btc[6:].index,name='pred_close')

In [ ]:
pred_nbeats=pd.concat([btc,pred],axis=1)

In [ ]:
pred_nbeats.to_csv('pred_nbeatsg.csv',index=False)